First execute the content of utils.py. Update if necessary.

In [17]:
pyscript.silence_start

''

In [1]:

try:
    # For the linter
    from pyscript_types import state, datetime
except ImportError:
    # When running in Jupyter
    pass

def set_state_bool(name: str, value: bool) -> None:
    if value:
        state.set(f"pyscript.{name}", 'true')
    else:
        state.set(f"pyscript.{name}", 'false')

def get_state_bool(name: str) -> bool:
    return state.get(f"pyscript.{name}") == 'true'

def set_state_datetime(name: str, value: datetime) -> None:
    state.set(f"pyscript.{name}", value.isoformat())

def set_state_none(name: str) -> None:
    state.set(f"pyscript.{name}", '')

def get_state_datetime(name: str) -> datetime | None:
    value = state.get(f"pyscript.{name}")
    if value != "":
      return datetime.fromisoformat(value)
    return None

def get_state_int(name: str) -> int:
    try:
        value = state.get(f"pyscript.{name}")
        if value != "":
            return int(value)
    except NameError:
        return 0
    return 0

def state_inc(name: str) -> None:
    value = get_state_int(name)
    state.set(f"pyscript.{name}", str(value + 1))

Then run any other file from pyscript folder

In [33]:
from datetime import datetime, timedelta
from typing import Any, Dict
import json

try:
    from .utils import state_inc, set_state_datetime, get_state_datetime, set_state_none
except ImportError:
    # In Jupyter, execute the content of utils.py in a cell first
    pass

try:
    # For the linter
    from pyscript_types import state, service, task, log, state_trigger, time_trigger, pyscript
except ImportError:
    # When running in Jupyter
    pass

# Constants
SWITCH_AFTER_LOW_POWER_MINUTES: int = 6
# will detect coffi made when above
COFFE_MAKING_POWER_LOWER = 800
# will detect coffi made when above for at least
COFFEE_MAKING_DURATION_MIN_SECONDS = 10

power_history = []

@mqtt_trigger("zigbee2mqtt/espresso_machine")
def update_power_average(payload=None):
    global power_history

    try:
        data = json.loads(payload)
        power = float(data.get("power", 0))
    except Exception as e:
        log.error(f"Failed to parse MQTT power payload: {e}")
        return

    now = datetime.now()

 
    power_history.append((now, power))
 
    cutoff = now - timedelta(minutes=5)
    power_history = [(ts, val) for ts, val in power_history if ts >= cutoff]

    if power_history:
        total = 0
        for _, val in power_history:
          total += val 
        pyscript.espresso_power_avg=int(total / len(power_history))
        log.info(f"☕ Power avg over 3min: {pyscript.espresso_power_avg}W from {len(power_history)} readings")


VAR_LAST_LOW_POWER: str = "last_low_power" 
set_state_none(VAR_LAST_LOW_POWER)

@state_trigger("pyscript.espresso_power_avg")
@time_trigger("period(minute=1)")  # Ensure periodic check in case of no new updates
def check_espresso_inactive(value=None):
    global last_low_power

    power = int(value)   
    now = datetime.now()


    if 1 < power < 400:
        last_low_power=get_state_datetime(VAR_LAST_LOW_POWER)
        if last_low_power is None: 
            set_state_datetime(VAR_LAST_LOW_POWER, now)     
        else:                   
            if now - last_low_power >= timedelta(minutes=SWITCH_AFTER_LOW_POWER_MINUTES):
                log.info(f"☕ Power {power}W below threshold for 6 min — cycling power")
                service.call("switch", "turn_off", entity_id="switch.espresso_machine")
                task.sleep(5)
                service.call("switch", "turn_on", entity_id="switch.espresso_machine")
                set_state_none(VAR_LAST_LOW_POWER)
    else:
        set_state_none(VAR_LAST_LOW_POWER)      


@state_trigger("sensor.espresso_machine_power | float > 800")
async def coffee_counter(value=None):
    task.unique("coffee_counter", kill_me=True)
    try:
        power = float(value)
    except (ValueError, TypeError):
        return

    log.info(f"☕ Power above 800W detected ({power}W) — waiting 10 seconds to confirm...")
    await task.sleep(COFFEE_MAKING_DURATION_MIN_SECONDS)

    # Re-check the power
    current_power = float(state.get("sensor.espresso_machine_power"))
    if current_power > 800:
        log.info(f"☕ Power still above {COFFE_MAKING_POWER_LOWER}W after {COFFEE_MAKING_DURATION_MIN_SECONDS}s — counting coffee")
        state_inc("espressos_today")
    else:
        log.info(f"❌ Power dropped to {current_power}W — no coffee counted")        

In [38]:
sensor.espresso_machine_power

'92'

In [37]:
get_state_int("espressos_today")

0

In [17]:


for eid in state.names():
        if "espre" in eid:
            value = state.get(eid)
            log.info(f"→ {eid}: [state: {value}]")


→ switch.espresso_machine: [state: on]
→ number.espresso_machine_countdown: [state: 0]
→ select.espresso_machine_power_outage_memory: [state: unknown]
→ select.espresso_machine_indicator_mode: [state: off/on]
→ sensor.espresso_machine_power: [state: 0]
→ sensor.espresso_machine_current: [state: 0]
→ sensor.espresso_machine_voltage: [state: 240]
→ sensor.espresso_machine_energy: [state: 3]
→ switch.espresso_machine_child_lock: [state: unknown]
→ update.espresso_machine: [state: off]
→ pyscript.espresso_power_avg: [state: 0]


In [20]:
from datetime import datetime, timedelta

def set_state_datetime(name: str, value: datetime) -> None:
    state.set(f"pyscript.{name}", value.isoformat())

def set_state_none(name: str) -> None:
    state.set(f"pyscript.{name}", '')

def get_state_datetime(name: str) -> datetime | None:
    try:
        value = state.get(f"pyscript.{name}")
        if value != "":
            return datetime.fromisoformat(value)
    except Exception as e:
        return None
    return None

In [27]:

s = get_state_datetime('x' )
s



datetime.datetime(2025, 5, 20, 20, 27, 57, 780506)

In [32]:
set_state_datetime('xx', datetime.now())

In [34]:
get_state_datetime('xx')

datetime.datetime(2025, 5, 20, 20, 39, 0, 995143)